In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager




print(plt.style.available)
%matplotlib inline
driver = webdriver.Chrome()

url='https://cq.zu.fang.com/house-a060-b014802.html'
driver.get(url)
driver.find_element(By.XPATH, '/html/body/div[4]/div[4]').click()

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [3]:
    list_container = driver.find_element(By.CSS_SELECTOR, '.houseList') 
    all_house_items = list_container.find_elements(By.TAG_NAME, 'dl') 

data = []


for item in all_house_items:
     try:
        # 假设房源信息（如标题、价格等）在卡片的不同子元素中
        
        title_element = item.find_element(By.CSS_SELECTOR, '.font15.mt12.bold')
        title = title_element.text
        
        price_element = item.find_element(By.CLASS_NAME, 'price') 
        price = price_element.text
        
        # 创建一行数据
        row_data = [title, price]
        data.append(row_data)
        
     except NoSuchElementException:
        # 如果某个房源卡片缺少某个字段（如价格），可以跳过或填空
        print("警告：某个房源卡片缺少完整数据。")
        continue

df = pd.DataFrame(data)


In [4]:
df

,0,1
0,整租|1室1厅|30㎡|朝东,1300
1,整租|3室2厅|106㎡|朝南,1350
2,整租|3室2厅|99㎡|朝南北,1200
3,整租|2室1厅|65㎡|朝东南,1700
4,整租|3室1厅|102㎡|朝东西,1300
5,整租|3室1厅|90㎡|朝南,800
6,整租|3室1厅|82㎡|朝南,1800
7,整租|2室1厅|62㎡|朝西南,1100
8,整租|1室1厅|50㎡|朝南,1200
9,整租|3室2厅|105㎡|朝南,1300


In [5]:
df['Size'] = df[0].astype(str).str.extract(r'(\d+\.?\d*)\s*㎡')
df['总价(元)'] = pd.to_numeric(df[1], errors='coerce')
df['面积(㎡)'] = pd.to_numeric(df['Size'], errors='coerce')
df['单价(元/㎡)']=df['总价(元)']/df['面积(㎡)']
df['单价(元/㎡)']=df['单价(元/㎡)'].round(0)
df

,0,1,Size,总价(元),面积(㎡),单价(元/㎡)
0,整租|1室1厅|30㎡|朝东,1300,30,1300,30,43.0
1,整租|3室2厅|106㎡|朝南,1350,106,1350,106,13.0
2,整租|3室2厅|99㎡|朝南北,1200,99,1200,99,12.0
3,整租|2室1厅|65㎡|朝东南,1700,65,1700,65,26.0
4,整租|3室1厅|102㎡|朝东西,1300,102,1300,102,13.0
5,整租|3室1厅|90㎡|朝南,800,90,800,90,9.0
6,整租|3室1厅|82㎡|朝南,1800,82,1800,82,22.0
7,整租|2室1厅|62㎡|朝西南,1100,62,1100,62,18.0
8,整租|1室1厅|50㎡|朝南,1200,50,1200,50,24.0
9,整租|3室2厅|105㎡|朝南,1300,105,1300,105,12.0


In [6]:
columns_to_drop = [0, 1, 'Size']
df_final = df.drop(columns_to_drop,axis=1)
df_final

,总价(元),面积(㎡),单价(元/㎡)
0,1300,30,43.0
1,1350,106,13.0
2,1200,99,12.0
3,1700,65,26.0
4,1300,102,13.0
5,800,90,9.0
6,1800,82,22.0
7,1100,62,18.0
8,1200,50,24.0
9,1300,105,12.0


In [15]:
 df_final.to_excel('陈家桥租价数据.xlsx', index=False)